# Training file

This file regroups every kind of test for model training : 
- The first file & library imports are mandatory
- The last exports are mandatory
- The model training depends on the choice of algorithm

There is an option to automate the training using togglable parameters 
Make sure to have set up the correct file structure using env_setup.ipynb

## Python libraries

In [ ]:
# Data Libraries
import json
import pandas as pd
import numpy as np
import os

# TensorFlow / Keras Part
import tensorflow as tf
from tensorflow import keras
from keras import backend
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Input, GlobalAveragePooling1D, Dropout
from keras.preprocessing import timeseries_dataset_from_array
import keras_tuner as kt

# SKLearn Part
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

## Parameters

### Code parameters

In [ ]:
# Basic constant file parameters
FILENAME="model_training.ipynb"
DIRNAME=os.path.abspath(FILENAME).replace(FILENAME,'')

# Datafile parameters
DATA_FOLDER = "versuch_f1"
VERSION_NB = 1
DF_POINTS_RANGE = 11
DF_POINTS_LENGTH = 3000

# Data parameters
OFFSET_XYZ=[-578.6,261.4,-375]
CARTESIAN_COLUMNS=['curCart_x','comdCart_x','curCart_y','comdCart_y','curCart_z','comdCart_z']

# Model parameters
MODEL_TYPE = "ffnn_regression"
TEST_TRAIN_RATIO = 0.1
VALID_TRAIN_RATIO = 0.2
MAX_TRIALS = 5 
MAX_EXECUTIONS = 3
MAX_EPOCHS = 50

### JSON Support

In [ ]:
def json_import(path):
    # Read JSON file
    with open(path,"r") as file :
        data = json.load(file)
    for variable in data :
        globals()[f'{variable}'] = data[variable]
    

## Import training DataFrame

### Import from Feather file

In [ ]:
def data_import():
    # Load DataFrame
    full_df=pd.read_feather(DIRNAME+"data/feather/"+DATA_FOLDER+"_"+str(DF_POINTS_LENGTH)+"_"+str(VERSION_NB)+".feather")

    # Drop unnecessary columns
    full_df=full_df.drop(["t_Sec","t_nSec","Fx1","Fy1","Fz1","Tx1","Ty1","Tz1"],axis=1)

    # Transform position from absolute to relative
    for i in range(len(CARTESIAN_COLUMNS)):
        full_df[CARTESIAN_COLUMNS[i]] = full_df[CARTESIAN_COLUMNS[i]] + OFFSET_XYZ[i//2]

    return full_df

### Split the dataset into train/validation sets

In [ ]:
def add_classes(df):
        for i in range(0,df.shape[0]):
            # Recovering cartesian positions
            x = df['curCart_x'][i]
            y = df['curCart_y'][i]
            # Calculating classes
            f1 = y - 2*x
            f2 = y + 2*x
            f3 = y - x/2
            f4 = y + x/2
            # class 1
            if (f1<0) & (f3>=0):
                df.loc[i,'position']='class 1'
            # class 2
            elif (f3<0) & (f4>=0):
                df.loc[i,'position']='class 2'
            # class 3
            elif (f4<0) & (f2>=0):
                df.loc[i,'position']='class 3'
            # class 4
            elif (f2<0) & (f1<=0):
                df.loc[i,'position']='class 4'
            # class 5
            elif (f1>0) & (f3<=0):
                df.loc[i,'position']='class 5'
            # class 6
            elif (f3>0) & (f4<=0):
                df.loc[i,'position']='class 6'
            # class 7
            elif (f4>0) & (f2<=0):
                df.loc[i,'position']='class 7'
            # class 8
            elif (f2>0) & (f1>=0):
                df.loc[i,'position']='class 8'
        return df

def time_series_transform(df):
    timeseries_df = timeseries_dataset_from_array(df, None, sequence_length=10,shuffle=False)
    el = []
    for element in timeseries_df:
        el.append(element.numpy())
    el_full = np.concatenate(el, axis=0)
    return el_full

In [ ]:
def data_split(df):

    # Shuffle DataFrame
    shuffled_df=df.sample(frac=1,random_state=1)

    # X & Y datasets

    if MODEL_TYPE == "ffnn_regression" :
        X = shuffled_df[['exT_A1','exT_A2','exT_A3','exT_A4','exT_A5','exT_A6','exT_A7',
                    'msT_A1','msT_A2','msT_A3','msT_A4','msT_A5','msT_A6','msT_A7',
                    'Fx','Fy','Fz','Tx','Ty','Tz']].to_numpy()
        # Regression Y
        Y = shuffled_df[['curCart_x','curCart_y','curCart_z']].to_numpy()



    elif MODEL_TYPE == "ffnn_classification" :
        X = shuffled_df[['exT_A1','exT_A2','exT_A3','exT_A4','exT_A5','exT_A6','exT_A7',
                    'msT_A1','msT_A2','msT_A3','msT_A4','msT_A5','msT_A6','msT_A7',
                    'Fx','Fy','Fz','Tx','Ty','Tz']].to_numpy()
        # Classification Y
        shuffled_df=add_classes(shuffled_df)
        Y_unbinarized = shuffled_df[['position']].to_numpy()

        # Classes to matrix using Binarizer
        binarizer = LabelBinarizer().fit(Y_unbinarized)
        Y = binarizer.transform(Y_unbinarized)


    elif MODEL_TYPE == "cnn_regression" :
        select_df = shuffled_df[['exT_A1','exT_A2','exT_A3','exT_A4','exT_A5','exT_A6','exT_A7',
                                 'msT_A1','msT_A2','msT_A3','msT_A4','msT_A5','msT_A6','msT_A7',
                                 'Fx','Fy','Fz','Tx','Ty','Tz','curCart_x','curCart_y','curCart_z','key']]
        # Split the dataset into smaller datasets based on key
        for i in range(0,DF_POINTS_RANGE):
            for j in range(0,DF_POINTS_RANGE):
                locals()[f'key{i}_{j}'] = select_df.loc[select_df['key'] == '{0},{1}'.format(i,j)]
                for k in range(0,20):
                    locals()[f'key{i}_{j}_{k}'] = time_series_transform(locals()[f'key{i}_{j}'].iloc[:,k])
        for i in range(0,DF_POINTS_RANGE):
            for j in range(0,DF_POINTS_RANGE):
                locals()[f'ts_key{i}_{j}'] = []
                for k in range(0,20):
                    locals()[f'ts_key{i}_{j}'].append(locals()[f'key{i}_{j}_{k}'])
                locals()[f'ts_key{i}_{j}'] = np.stack(locals()[f'ts_key{i}_{j}'], axis=-1)
        # Define Input
        X = []
        for i in range(0,DF_POINTS_RANGE):
            for j in range(0,DF_POINTS_RANGE):
                X.append(locals()[f'ts_key{i}_{j}'])
        X = np.concatenate(X, axis=0)
        # Define Output
        Y = []
        for i in range(0,DF_POINTS_RANGE):
            for j in range(0,DF_POINTS_RANGE):
                Y.append(locals()[f'key{i}_{j}'][['curCart_x','curCart_y']][9:].to_numpy())
        Y = np.concatenate(Y, axis=0)



    elif MODEL_TYPE == "cnn_classification" :
        shuffled_df=add_classes(shuffled_df)
        select_df = shuffled_df[['exT_A1','exT_A2','exT_A3','exT_A4','exT_A5','exT_A6','exT_A7',
                                 'msT_A1','msT_A2','msT_A3','msT_A4','msT_A5','msT_A6','msT_A7',
                                 'Fx','Fy','Fz','Tx','Ty','Tz','curCart_x','curCart_y','curCart_z','key','position']]
        # Split the dataset into smaller datasets based on key
        for i in range(0,DF_POINTS_RANGE):
            for j in range(0,DF_POINTS_RANGE):
                locals()[f'key{i}_{j}'] = select_df.loc[select_df['key'] == '{0},{1}'.format(i,j)]
                for k in range(0,20):
                    locals()[f'key{i}_{j}_{k}'] = time_series_transform(locals()[f'key{i}_{j}'].iloc[:,k])
        for i in range(0,DF_POINTS_RANGE):
            for j in range(0,DF_POINTS_RANGE):
                locals()[f'ts_key{i}_{j}'] = []
                for k in range(0,20):
                    locals()[f'ts_key{i}_{j}'].append(locals()[f'key{i}_{j}_{k}'])
                locals()[f'ts_key{i}_{j}'] = np.stack(locals()[f'ts_key{i}_{j}'], axis=-1)
        # Define Input
        X = []
        for i in range(0,DF_POINTS_RANGE):
            for j in range(0,DF_POINTS_RANGE):
                X.append(locals()[f'ts_key{i}_{j}'])
        X = np.concatenate(X, axis=0)
        # Define Output
        Y = []
        for i in range(0,DF_POINTS_RANGE11):
            for j in range(0,DF_POINTS_RANGE):
                Y.append(locals()[f'key{i}_{j}'][['position']][9:].to_numpy())
        Y_unbinarized = np.concatenate(Y, axis=0)

        # Classes to matrix using Binarizer
        binarizer = LabelBinarizer().fit(Y_unbinarized)
        Y = binarizer.transform(Y_unbinarized)



    else :
        raise ValueError("MODEL_TYPE parameter is not valid")


    # Train / Test / Validation dataset
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=TEST_TRAIN_RATIO, random_state=2)
    X_Train, X_Valid, Y_Train, Y_Valid = train_test_split(X_Train, Y_Train, test_size=VALID_TRAIN_RATIO, random_state=3)

    return X_Train,X_Test,X_Valid,Y_Train,Y_Test,Y_Valid

## Model building

### Model 1 - FFNN Regression

In [ ]:
def rmse(y_true,y_pred):    # Private
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
def rsquared(y_true,y_pred):    # Private
    RSS =  backend.sum(backend.square( y_true- y_pred ))
    TSS = backend.sum(backend.square( y_true - backend.mean(y_true)))
    return 1-(RSS/TSS)

class FFNN_Regression_Model(kt.HyperModel):

    # Search metrics
    objective="mae"

    def build(self, hp):    # Main
        # Create model
        model = Sequential()

        # Input Layer
        model.add(Dense(20, input_shape=(20,), activation='relu'))

        # Tune number of layers
        for i in range(hp.Int("num_layers", 1, 5)):
            model.add(Dense(
                    # Tune number of units
                    units = hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                    # Tune activation function
                    activation = hp.Choice("activation", ["relu","tanh"])
                            )
                    )
        # Tune whether to use dropout
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=0.2))

        # Output Layer
        model.add(Dense(3, activation='linear'))

        # Define the optimizer learning rate as hyperparameter
        lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        # Compile model
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae",rmse,rsquared])
        return model

    # Custom fit function
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size", [16, 32, 64]),**kwargs,)

### Model 2 - CNN Regression

In [ ]:
def rmse(y_true,y_pred):        # Private
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
def rsquared(y_true,y_pred):    # Private
    RSS =  backend.sum(backend.square( y_true- y_pred ))
    TSS = backend.sum(backend.square( y_true - backend.mean(y_true)))
    return 1-(RSS/TSS)

class CNN_Regression_Model(kt.HyperModel):

    # Search metrics
    objective="mae"

    def build(self, hp):
        # Create model
        model = Sequential()
        
        # Input convolutional Layer
        model.add(Conv1D(
                    # input shape
                    input_shape = (10,20),
                    # filters
                    filters = hp.Int('conv_input_filter', min_value=16, max_value=128, step=16),
                    # kernel_size
                    kernel_size = hp.Choice('conv_input_kernel', values=[3,5]),
                    # activation
                    activation = hp.Choice('conv_input_activation', ['relu','tanh']),
                    padding="same"
                ))
        
        # Hidden convolutional Layers
        for i in range (hp.Int("num_layers",1,6)):
            model.add(Conv1D(
                        # filters
                        filters = hp.Int(f'conv_{i}_filter', min_value=16, max_value=128, step=16),
                        # kernel_size
                        kernel_size = hp.Choice(f'conv_{i}_kernel', values=[3,5]),
                        # activation
                        activation = hp.Choice(f'conv_{i}_activation', ['relu','tanh']),
                        padding="same"
                    ))
        
        # Tune whether to use dropout
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=0.25))
        
        # Pooling Layer
        model.add(GlobalAveragePooling1D())
        
        # Output Layer
        model.add(Dense(2, activation='linear'))
        
        # Define the optimizer learning rate as hyperparameter
        lr = hp.Float("lr", min_value=1e-3, max_value=1e-2, sampling="log")
        # Compile model
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                    loss="mse", 
                    metrics=["mae",rmse,rsquared])
        return model

    # Custom fit function
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size", [16, 32, 64]),**kwargs,)

### Model 3 - FFNN Classification

In [ ]:
class FFNN_Classification_Model(kt.HyperModel):

    # Search metrics
    objective="accuracy"

    def build(self, hp):
        # Create model
        model = Sequential()

        # Input Layer
        model.add(Dense(20, input_shape=(20,), activation='relu'))

        # Hidden Layers
        for i in range(hp.Int("num_layers", 1, 5)):
            model.add(Dense(
                    # Tune number of units
                    units = hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                    # Tune activation function
                    activation = hp.Choice("activation", ["relu","tanh"])
                            )
                     )
        # Tune whether to use dropout
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=0.2))

        # Output Layer
        model.add(Dense(8, activation='softmax'))

        # Define the optimizer learning rate as hyperparameter
        lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        # Compile model
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                      loss="categorical_crossentropy", 
                      metrics=["accuracy"])
        return model

    # Custom fit function
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size", [16, 32, 64]),**kwargs,)

### Model 4 - CNN Classification

In [ ]:
class CNN_Classification_Model(kt.HyperModel):

    # Search metrics
    objective="accuracy"

    def build(self, hp):
        # Create model
        model = Sequential()
        
        # Input convolutional Layer
        model.add(Conv1D(
                    # input shape
                    input_shape = (10,20),
                    # filters
                    filters = hp.Int('conv_input_filter', min_value=16, max_value=128, step=16),
                    # kernel_size
                    kernel_size = hp.Choice('conv_input_kernel', values=[3,5]),
                    # activation
                    activation = hp.Choice('conv_input_activation', ['relu','tanh']),
                    padding="same"
                ))
        
        # Hidden convolutional Layer
        for i in range (hp.Int("num_layers",1,6)):
            model.add(Conv1D(
                        # filters
                        filters = hp.Int(f'conv_{i}_filter', min_value=16, max_value=128, step=16),
                        # kernel_size
                        kernel_size = hp.Choice(f'conv_{i}_kernel', values=[3,5]),
                        # activation
                        activation = hp.Choice(f'conv_{i}_activation', ['relu','tanh']),
                        padding="same"
                    ))
        
        # Tune whether to use dropout
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=0.25))
        
        # Pooling Layer
        model.add(GlobalAveragePooling1D())
        
        # Output Layer
        model.add(Dense(8, activation='softmax'))
        
        # Define the optimizer learning rate as hyperparameter
        lr = hp.Float("lr", min_value=1e-3, max_value=1e-2, sampling="log")
        # Compile model
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                    loss="categorical_crossentropy", 
                    metrics=["accuracy"])
        return model

    # Custom fit function
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size", [16, 32, 64]),**kwargs,)

## Hyperparameter optimization

### Algorithm 1 - RandomSearch

In [ ]:
def KT_RandomSearch(Selected_Model):
    # Keras search function
    tuner = kt.RandomSearch(
        Selected_Model,
        objective=Selected_Model.objective,
        max_trials=MAX_TRIALS,
        executions_per_trial=MAX_EXECUTIONS,
        overwrite=True,
        directory=DIRNAME+"/results/search/"+MODEL_TYPE,
        project_name=DATA_FOLDER+"_"+str(DF_POINTS_LENGTH)+"_"+str(VERSION_NB)
    )

    return tuner

### Algorithm 2 - GridSearch

In [ ]:
def KT_GridSearch(Selected_Model):
    # Keras search function
    tuner = kt.GridSearch(
        Selected_Model,
        objective=Selected_Model.objective,
        max_trials=MAX_TRIALS,
        overwrite=True,
        tune_new_entries=True,
        allow_new_entries=True,
        directory=DIRNAME+"/results/search/"+MODEL_TYPE,
        project_name=DATA_FOLDER+"_"+str(DF_POINTS_LENGTH)+"_"+str(VERSION_NB)
    )

    return tuner

### Algorithm 3 - Bayesian Optimizer

In [ ]:
def KT_BayesianSearch(Selected_Model):
    # Keras search function
    tuner = kt.BayesianOptimization(
        Selected_Model,
        objective=Selected_Model.objective,
        max_trials=MAX_TRIALS,
        alpha=0.0001,
        beta=2.6,
        overwrite=True,
        tune_new_entries=True,
        allow_new_entries=True,
        directory=DIRNAME+"/results/search/"+MODEL_TYPE,
        project_name=DATA_FOLDER+"_"+str(DF_POINTS_LENGTH)+"_"+str(VERSION_NB)
    )

    return tuner

## Global Search

In [ ]:
# JSON directory scan
CONFIG_DIR=DIRNAME+"config"
JSON_files = [file for file in os.listdir(CONFIG_DIR) if file.endswith(".json")]
JSON_files.sort()

counter = 3

for JSON_file in JSON_files :
    # Import
    json_import(CONFIG_DIR+"/"+JSON_file)
    print(">> File Imported")
    # Prepare
    data_df = data_import()
    # Split
    X_Train,X_Test,X_Valid,Y_Train,Y_Test,Y_Valid=data_split(data_df)
    print(">> Dataset Split")

    # Model selector
    if (MODEL_TYPE == "ffnn_regression") :
        model = FFNN_Regression_Model()
    elif (MODEL_TYPE == "ffnn_classification") :
        model = FFNN_Classification_Model()
    elif (MODEL_TYPE == "cnn_regression") :
        model = CNN_Regression_Model()
    elif (MODEL_TYPE == "cnn_classification") :
        model = CNN_Classification_Model()

    print(">> Model Loaded")

    tuners_list = [KT_RandomSearch(model),KT_GridSearch(model),KT_BayesianSearch(model)]

    print(Y_Test[0])

    for i in range(len(tuners_list)) :
        tuner = tuners_list[i]
        print(">> Tuner Searching")
        # Search
        tuner.search(X_Train, Y_Train, epochs=15, validation_data=(X_Valid, Y_Valid),callbacks=[keras.callbacks.TensorBoard('results/tensorboard/')])
        # Export results
        best_hps=tuner.get_best_hyperparameters()[0]


        # Save Results
        with open(DIRNAME+"results/models/"+DATA_FOLDER+"_"+MODEL_TYPE+"_"+str(VERSION_NB)+"_"+str(counter)+".obj","w") as result_file :
            json.dump(best_hps.values,result_file)

        counter+=1

    print("Program Successful !")

## Train with set Hyperparameters

In [ ]:
def build_ffnn_regression(hp):    # Main
        # Create model
        model = Sequential()

        # Input Layer
        model.add(Dense(20, input_shape=(20,), activation='relu'))

        # Tune number of layers
        for i in range(hp.Int("num_layers", 1, 5)):
            model.add(Dense(
                    # Tune number of units
                    units = hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                    # Tune activation function
                    activation = hp.Choice("activation", ["relu","tanh"])
                            )
                    )
        # Tune whether to use dropout
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=0.2))

        # Output Layer
        model.add(Dense(3, activation='linear'))

        # Define the optimizer learning rate as hyperparameter
        lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        # Compile model
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae",rmse,rsquared])
        return model

path =
hp = kt.HyperParameters()

with open(path,"r") as file :
    hp.values = json.load(file)

json_import(DIRNAME+"config/test1_0.json")
df = data_import()
X_Train,X_Test,X_Valid,Y_Train,Y_Test,Y_Valid=data_split(df)

final_model = build_ffnn_regression(hp)
final_model.fit(X_Train,Y_Train,epochs=40,verbose=1,validation_data=[X_Valid,Y_Valid])